In [ ]:
# GAE

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges, negative_sampling
import pandas as pd
import uuid
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, average_precision_score

from ast import literal_eval
import numpy as np

from torch_geometric.nn import Node2Vec

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Load datasets
job_descriptions = pd.read_csv('./data/processed/job_descriptions_processed.csv')
resumes = pd.read_csv('./data/processed/general-resume-dataset-processed-v1.csv', converters={'skills': literal_eval})

# Shuffle job_descriptions and select the first N rows
job_descriptions = job_descriptions.sample(frac=1, random_state=random_seed).head(50000)

# Convert 'skills' column to list
job_descriptions['skills'] = job_descriptions['skills'].apply(literal_eval)

In [ ]:
# Replace None values in job_title and category with a default value before encoding
job_descriptions['job_title'].fillna('unknown', inplace=True)
resumes['job_title'].fillna('unknown', inplace=True)
resumes['category'].fillna('unknown', inplace=True)

# Ensure job_ids and candidate_ids are correctly assigned
job_descriptions['job_id'] = range(1, len(job_descriptions) + 1)
resumes['candidate_id'] = range(1, len(resumes) + 1)

# Add 'unknown' to the list of all titles and categories to handle unseen labels
all_titles = job_descriptions['job_title'].tolist() + resumes['job_title'].tolist()
all_titles.append('unknown')
all_categories = resumes['category'].tolist()
all_categories.append('unknown')

In [ ]:
# Fit the label encoders
le_job_title = LabelEncoder()
le_category = LabelEncoder()
le_job_title.fit(all_titles)
le_category.fit(all_categories)

# Transform the columns
job_descriptions['job_title'] = le_job_title.transform(job_descriptions['job_title'])
resumes['job_title'] = le_job_title.transform(resumes['job_title'])
resumes['category'] = le_category.transform(resumes['category'])

# Encode skills
all_skills = set(skill for skills in job_descriptions['skills'].tolist() + resumes['skills'].tolist() for skill in skills)
le_skills = {skill: i for i, skill in enumerate(all_skills)}

In [ ]:
# Create nodes and edges for the graph
nodes = []
edges = []
weights = []
node_features = []

jobs_from_edges = []
candidates_from_edges = []
jobs_and_candidates_from_edges = []

skill_weight_multiplier = 1  # Weight for skill overlap
title_weight = 5  # Weight for job title match

# Add job nodes
for i, row in job_descriptions.iterrows():
    nodes.append(row['job_id'])
    skills_vector = [0] * len(le_skills)
    if row['skills']:  # Check if skills are not empty
        for skill in row['skills']:
            skills_vector[le_skills[skill]] = 1
    node_features.append([row['job_title']] + skills_vector)

# Add resume nodes, using 'category' instead of 'job_title'
for i, row in resumes.iterrows():
    nodes.append(row['candidate_id'] + len(job_descriptions))
    skills_vector = [0] * len(le_skills)
    if row['skills']:  # Check if skills are not empty
        for skill in row['skills']:
            skills_vector[le_skills[skill]] = 1
    node_features.append([row['job_title']] + skills_vector)

In [ ]:
def skills_overlap(job_skills, resume_skills):
    return len(set(job_skills).intersection(set(resume_skills)))

threshold = 1  # Set a threshold for the minimum number of overlapping skills

for i, job in job_descriptions.iterrows():
    for j, resume in resumes.iterrows():
        overlap = skills_overlap(job['skills'], resume['skills'])
        combined_weight = overlap * skill_weight_multiplier

        if job['job_title'] == resume['job_title']:
            combined_weight += title_weight  # Add weight for job title match

        if combined_weight > 0:
            edges.append((job['job_id'], resume['candidate_id'] + len(job_descriptions)))
            weights.append(combined_weight)
            jobs_and_candidates_from_edges.append((job['job_id'], resume['candidate_id']))
            jobs_from_edges.append(job['job_id'])
            candidates_from_edges.append(resume['candidate_id'])

nodes_length = len(nodes)

In [ ]:
# pd.DataFrame(node_features).to_csv('./data/features/raw/nodes.csv')
# pd.DataFrame(node_features).to_csv('./data/features/raw/node_features.csv')
# pd.DataFrame(edges).to_csv('./data/features/raw/edges.csv')
# pd.DataFrame(weights).to_csv('./data/features/raw/weights.csv')

In [ ]:
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
edge_weight = torch.tensor(weights, dtype=torch.float)
edge_index = edge_index.clamp(0, nodes_length - 1)

In [ ]:
torch.save(edge_index, './data/features/edge_index-50k.pt')
torch.save(edge_weight, './data/features/edge_weight-50k.pt')
with open('./data/features/nodes_len.txt', 'w') as file:
    file.write(str(len(nodes)))

In [ ]:
# edge_index = torch.load('./data/features/edge_index-50k.pt')
# edge_weight = torch.load('./data/features/edge_weight-50k.pt')
# nodes_length = 0
# with open('./data/features/nodes_len.txt', 'r') as file:
#     nodes_length = np.int64(file.read())

In [ ]:
# jobs_from_edges = set(jobs_from_edges)
# candidates_from_edges = set(candidates_from_edges)
# jobs_and_candidates_from_edges = set(jobs_and_candidates_from_edges)

In [ ]:
# pd.DataFrame(jobs_from_edges).to_csv('./data/features/raw/jobs_from_edges.csv')
# pd.DataFrame(candidates_from_edges).to_csv('./data/features/raw/candidates_from_edges.csv')
# pd.DataFrame(jobs_and_candidates_from_edges).to_csv('./data/features/raw/jobs_and_candidates_from_edges.csv')

In [ ]:
# Convert node features to tensor
x = torch.tensor(node_features, dtype=torch.float)

In [ ]:
torch.save(x, './data/features/x-50k.pt')

In [ ]:
# x = torch.load('./data/features/x-50k.pt')

In [ ]:
# Create PyTorch Geometric data object
data = Data(x=x, edge_index=edge_index, edge_weight=edge_weight)

# Ensure edge indices are within range
# data.num_nodes = nodes_length

original_edge_index = data.edge_index.clone()
original_edge_weight = data.edge_weight.clone()

# Splitting edges for training/validation
data = train_test_split_edges(data)

# Create a dictionary to map edge indices to their weights
edge_weight_dict = {tuple(edge_index[:, i].tolist()): edge_weight[i].item() for i in range(edge_index.size(1))}

def get_edge_weights(edge_index, edge_weight_dict):
    weights = []
    for i in range(edge_index.size(1)):
        edge = tuple(edge_index[:, i].tolist())
        weight = edge_weight_dict.get(edge, 0)  # Default to 0 if edge not found
        weights.append(weight)
    return torch.tensor(weights, dtype=torch.float)

In [ ]:
train_edge_weights = get_edge_weights(data.train_pos_edge_index, edge_weight_dict)
test_edge_weights = get_edge_weights(data.test_pos_edge_index, edge_weight_dict)
val_edge_weights = get_edge_weights(data.val_pos_edge_index, edge_weight_dict)

In [ ]:
data.train_pos_edge_weight = train_edge_weights
data.test_pos_edge_weight = test_edge_weights
data.val_pos_edge_weight = val_edge_weights

# Manually create negative edges for training
neg_edge_index_train = negative_sampling(
    edge_index=data.train_pos_edge_index,
    num_nodes=data.num_nodes,
    num_neg_samples=data.train_pos_edge_index.size(1),
)
data.train_neg_edge_index = neg_edge_index_train

# Assign zero weights to negative edges for training
neg_train_edge_weights = torch.zeros(neg_edge_index_train.size(1), dtype=torch.float)

# Manually create negative edges for testing
neg_edge_index_test = negative_sampling(
    edge_index=data.test_pos_edge_index,
    num_nodes=data.num_nodes,
    num_neg_samples=data.test_pos_edge_index.size(1),
)
data.test_neg_edge_index = neg_edge_index_test

# Assign zero weights to negative edges for testing
neg_test_edge_weights = torch.zeros(neg_edge_index_test.size(1), dtype=torch.float)

# Combine positive and negative edge weights for training
data.train_neg_edge_weight = neg_train_edge_weights

# Combine positive and negative edge weights for testing
data.test_neg_edge_weight = neg_test_edge_weights

# Ensure edge_index tensors are of integer type
data.train_pos_edge_index = data.train_pos_edge_index.long()
data.test_pos_edge_index = data.test_pos_edge_index.long()
data.train_neg_edge_index = data.train_neg_edge_index.long()
data.test_neg_edge_index = data.test_neg_edge_index.long()

In [ ]:
print(original_edge_index.shape)
print(original_edge_weight.shape)
print(data.train_pos_edge_index.shape)
print(train_edge_weights.shape)

In [ ]:
# Define number of nodes
num_nodes = data.num_nodes

# Node2Vec parameters
embedding_dim = 64
walk_length = 20
context_size = 10
walks_per_node = 10
batch_size = 128
lr = 0.01
num_epochs = 21

In [ ]:
# Initialize Node2Vec
node2vec = Node2Vec(
    edge_index=original_edge_index,
    embedding_dim=embedding_dim,
    walk_length=walk_length,
    context_size=context_size,
    walks_per_node=walks_per_node,
    num_negative_samples=1,
    p=1,
    q=1,
    sparse=True,
    num_nodes=num_nodes  # Specify number of nodes
)

# Move to the appropriate device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
node2vec = node2vec.to(device)

In [ ]:
# Optimizer for Node2Vec
optimizer = torch.optim.SparseAdam(node2vec.parameters(), lr=lr)

In [ ]:
# Training loop for Node2Vec
def train_node2vec(num_epochs):
    node2vec.train()
    for epoch in range(num_epochs):
        total_loss = 0
        loader = node2vec.loader(batch_size=batch_size, shuffle=True, num_workers=0)  # Set num_workers=0
        for i, (pos_rw, neg_rw) in enumerate(loader):
            optimizer.zero_grad()
            loss = node2vec.loss(pos_rw.to(device), neg_rw.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if epoch % 10 == 0:
                print(f'Epoch {epoch + 1}, Iteration {i}, Loss: {total_loss / 10}')
                total_loss = 0

In [ ]:
# Train Node2Vec model
train_node2vec(num_epochs)

In [ ]:
# Extract embeddings
node_embeddings = node2vec.embedding.weight.data.cpu().numpy()

# Update node features with embeddings
data.x = torch.tensor(node_embeddings, dtype=torch.float)

In [ ]:
class GAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAE, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

    def encode(self, x, edge_index, edge_weight):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        return self.conv2(x, edge_index, edge_weight)

    def decode(self, z, pos_edge_index, neg_edge_index):
        pos_pred = (z[pos_edge_index[0].long()] * z[pos_edge_index[1].long()]).sum(dim=1)
        neg_pred = (z[neg_edge_index[0].long()] * z[neg_edge_index[1].long()]).sum(dim=1)
        return pos_pred, neg_pred

    def forward(self, data):
        z = self.encode(data.x, data.train_pos_edge_index, data.train_pos_edge_weight)
        return z

In [ ]:
# Initialize and train GAE model as before
model = GAE(data.num_node_features, 16)  # Adjust dimensions as needed
gae_optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
def train(data):
    model.train()
    gae_optimizer.zero_grad()
    z = model.encode(data.x, data.train_pos_edge_index, data.train_pos_edge_weight)  # Pass train_pos_edge_weight
    pos_pred, neg_pred = model.decode(z, data.train_pos_edge_index, data.train_neg_edge_index)
    pos_loss = loss_fn(pos_pred, torch.ones_like(pos_pred))
    neg_loss = loss_fn(neg_pred, torch.zeros_like(neg_pred))
    loss = pos_loss + neg_loss
    if torch.isnan(loss) or torch.isinf(loss):
        print("Warning: NaN or Inf loss detected")
        return float('inf')
    loss.backward()
    gae_optimizer.step()
    return loss.item()

In [ ]:
for epoch in range(1000):
    loss = train(data)
    if loss == float('inf'):
        break
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')

In [ ]:
def evaluate_model(data, model):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.val_pos_edge_index, data.val_pos_edge_weight)
        pos_pred = torch.sigmoid((z[data.val_pos_edge_index[0].long()] * z[data.val_pos_edge_index[1].long()]).sum(dim=1)).cpu().numpy()
        neg_pred = torch.sigmoid((z[data.val_neg_edge_index[0].long()] * z[data.val_neg_edge_index[1].long()]).sum(dim=1)).cpu().numpy()

    y_true = np.concatenate([np.ones(pos_pred.shape[0]), np.zeros(neg_pred.shape[0])])
    y_pred = np.concatenate([pos_pred, neg_pred])

    auc_roc = roc_auc_score(y_true, y_pred)
    ap = average_precision_score(y_true, y_pred)

    return auc_roc, ap

In [ ]:
# Example usage
auc_roc, ap = evaluate_model(data, model)
print(f"AUC-ROC: {auc_roc:.4f}, AP: {ap:.4f}")

In [ ]:
def predict_best_candidates(job_descriptions, resumes, z):
    job_ids = job_descriptions['job_id'].values
    candidate_ids = resumes['candidate_id'].values + len(job_descriptions)

    predictions = []
    for job_id, job_title, job_skills in zip(job_ids, job_descriptions['job_title'], job_descriptions['skills']):
        best_match = None
        best_score = float('-inf')
        for candidate_id, category, candidate_job_title, skills in zip(
                resumes['candidate_id'], resumes['category'], resumes['job_title'], resumes['skills']):
            candidate_index = candidate_id + len(job_descriptions) - 1
            job_index = job_id - 1

            # Ensure indices are within range
            if candidate_index >= z.size(0) or job_index >= z.size(0):
                continue

            score = torch.sigmoid(torch.dot(z[job_index], z[candidate_index])).item()
            if score > best_score:
                best_score = score
                best_match = (job_id, job_title, candidate_id, candidate_job_title, category, skills, score)

        if best_match:
            job_id, job_title, candidate_id, candidate_job_title, category, skills, score = best_match
            match_percentage = score * 100  # Assuming the score is between 0 and 1
            predictions.append({
                "Job ID": job_id,
                "Job Title": le_job_title.inverse_transform([job_title])[0],
                "Candidate ID": candidate_id,
                "Candidate Job Title": le_job_title.inverse_transform([candidate_job_title])[0],
                "Candidate Category": le_category.inverse_transform([category])[0],
                "Match Percentage": match_percentage,
                "Mutual Skills": set(job_skills).intersection(set(skills)),
                "Job Skills": job_skills,
                "Candidate Skills": skills
            })

    predictions_df = pd.DataFrame(predictions)
    return predictions_df

In [ ]:
# Example usage
with torch.no_grad():
    z = model.encode(data.x, original_edge_index, data.edge_weight)

In [ ]:
jobs_to_predict = job_descriptions[job_descriptions['job_title'].isin(resumes['job_title'])].sample(frac=1, random_state=random_seed)[:100]

In [ ]:
predictions_df = predict_best_candidates(jobs_to_predict, resumes, z)

In [ ]:
predictions_df.head(100)

In [ ]:
torch.save(model.state_dict(), f"./models/gea-recommendation-system-50k-{auc_roc:.2f}-acc-{uuid.uuid4()}-{time.strftime('%Y%m%d-%H%M%S')}-v4.pth")